**Algorithm 1**
Hamiltonian Monte Carlo

**function** Leapfrog $(\theta,r,\epsilon)$

Set $\tilde{r} \leftarrow r + \frac{\epsilon}{2} \nabla_\theta \mathcal{L}(\theta)$

Set $\tilde{\theta} \leftarrow \theta + \epsilon \tilde{r}$

Set $\tilde{r} \leftarrow r + \frac{\epsilon}{2} \nabla_\theta \mathcal{L}(\tilde{\theta})$

**return** $\tilde{\theta},\tilde{r}$

$\mathcal{L}$ is the logarithm of the joint density of the variables of interest $\theta$. The Leapfrog function of Algorithm 1 implements the Stormer-Verlet ("leapfrog") integrator, which proceeds according to the updates:

$\tilde{r} \leftarrow r + \frac{\epsilon}{2} \nabla_\theta \mathcal{L}(\theta)$
